#TF-IDF


## Imports & Downloads

In [2]:
from datasets import load_dataset
import pandas as pd

import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download("wordnet")

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## Cargar Datos

In [3]:
dataset = load_dataset("wilfredomartel/small-spanish-legal-dataset", split="train")
df = pd.DataFrame(dataset)
for index, row in df.head().iterrows():
    print(f"<----------- Fila {index} ----------->")
    print(f"Query: {row['query']}")
    print(f"Positive: {row['pos'][0][:150]}...\n")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

small-spanish-legal-dataset.jsonl:   0%|          | 0.00/38.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11246 [00:00<?, ? examples/s]

<----------- Fila 0 ----------->
Query: ¿Cuál fue la razón principal por la que la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N° 2850-19-EP, presentada por Javier Jefferson Delgado Guano y otros contra la sentencia de la Sala Única Multicompetente de la Corte Provincial de Esmeraldas?
Positive: La Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N° 2850-19-EP debido a que los accionantes no presentaron un argume...

<----------- Fila 1 ----------->
Query: ¿Por qué la Sala de Admisión de la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N°. 3090-19-EP presentada por José Antonio Guzñay contra la sentencia de la Corte Provincial de Zamora?
Positive: La Sala de Admisión de la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N°. 3090-19-EP, presentada por el señor José...

<----------- Fila 2 ----------->
Query: ¿Cuál fue la controversia principal en el caso N

## Preprocesamiento

In [ ]:
stemmer = nltk.SnowballStemmer("spanish")
stop_words_es = set(stopwords.words("spanish"))

def preprocesar_texto(texto):
    texto_minusculas = texto.lower()
    tokens = word_tokenize(texto_minusculas, language='spanish')
    tokens_limpios = []
    for palabra in tokens:
        if palabra.isalpha():
            if palabra not in stop_words_es:
                palabra_raiz = stemmer.stem(palabra)
                tokens_limpios.append(palabra_raiz)
    return " ".join(tokens_limpios)

corpus_original = df['pos'].explode().dropna().unique()
corpus_limpio = [preprocesar_texto(texto) for texto in corpus_original]

print("<---- Prueba de Preprocesamiento ---->")
print(f"Texto Original:\n{corpus_original[0][:200]}...\n")
print(f"Texto Procesado:\n{corpus_limpio[0][:200]}...")

<---- Prueba de Preprocesamiento ---->
Texto Original:
La Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N° 2850-19-EP debido a que los accionantes no presentaron un argumento claro sobre los derechos supuestamente violado...

Texto Procesado:
cort constitucional inadmit tramit accion extraordinari proteccion deb accion present argument clar derech supuest viol relacion direct inmediat accion u omision sal unic multicompetent cort provincia...


## TF-IFD

In [ ]:
vectorizer = TfidfVectorizer()
matriz_tfidf = vectorizer.fit_transform(corpus_limpio)
print("Dimensiones TF-IDF:", matriz_tfidf.shape) #total de documentos #palabras únicas

Dimensiones TF-IDF: (22458, 11625)


## Búsqueda

In [ ]:
def buscar_tfidf(pregunta_usuario, top_n=5):
    print(f"<--- Buscando: '{pregunta_usuario}' --->")
    pregunta_procesada = preprocesar_texto(pregunta_usuario)
    pregunta_vector = vectorizer.transform([pregunta_procesada])
    similitudes = cosine_similarity(pregunta_vector, matriz_tfidf).flatten()
    indices_top = similitudes.argsort()[-top_n:][::-1]
    print("Resultados encontrados:\n")
    for i in indices_top:
        print(f"  Puntaje de Similitud: {similitudes[i]:.4f}")
        print(f"  Texto: {corpus_original[i][:250]}...\n")

pregunta_de_prueba = df.iloc[20]['query']

In [ ]:
buscar_tfidf(pregunta_de_prueba)

<--- Buscando: '¿Cuál fue la razón principal por la que la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N. 2511-18-EP presentada por Juan Ricardo Cedillo Rizo?' --->
Resultados encontrados:

  Puntaje de Similitud: 0.6206
  Texto: El recurso de apelación interpuesto por Juan Ricardo Cedillo Rizo contra la sentencia condenatoria fue desestimado por la Sala de lo Penal de la Corte Provincial de Justicia de Manabí el 09 de mayo de 2018. Esta decisión mantuvo incólume la sentencia...

  Puntaje de Similitud: 0.6184
  Texto: La Sala de lo Penal de la Corte Provincial de Justicia de Manabí, mediante fallo dictado el 09 de mayo del 2018, desechó el recurso de apelación presentado por el señor Juan Ricardo Cedillo Rizo. En dicha resolución, la Sala confirmó en todas sus par...

  Puntaje de Similitud: 0.4594
  Texto: El señor Juan Ricardo Cedillo Rizo fue procesado y posteriormente condenado por el delito de asociación ilícita, conforme a lo establecido en lo

In [ ]:
buscar_tfidf("¿Qué pasa con las obligaciones del vendedor?")

<--- Buscando: '¿Qué pasa con las obligaciones del vendedor?' --->
Resultados encontrados:

  Puntaje de Similitud: 0.3872
  Texto: Tras el contrato de compraventa, se inició un litigio donde Austrocorp S.A. demandó una reducción del precio por supuesta menor cabida del terreno vendido. En contrapartida, los vendedores reconvinieron exigiendo el pago de los $150.000,00 adeudados ...

  Puntaje de Similitud: 0.1940
  Texto: La decisión de la Sala de Admisión de la Corte Constitucional de admitir a trámite la acción extraordinaria de protección No. 1871-10-EP tiene una implicación fundamental: significa que la Corte ha considerado que la demanda cumple con los presupuest...

  Puntaje de Similitud: 0.1751
  Texto: La accionante, Ana Lucía Romero Medina de Santos, fundamentó su acción extraordinaria de protección en la presunta vulneración de dos derechos constitucionales fundamentales por parte del Juez Primero de Inquilinato de Guayaquil en el proceso de desa...

  Puntaje de Similitud:

# TRANSFORMERS

## Imports

In [4]:
import numpy as np
import os
from sentence_transformers import SentenceTransformer, util

## Modelo

In [5]:
#paraphrase-multilingual-MiniLM-L12-v2: incluye el español

modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Datos

In [6]:
corpus_original = df['pos'].explode().dropna().unique()
print(f"Texto Original:\n{corpus_original[0][:200]}...\n")

Texto Original:
La Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N° 2850-19-EP debido a que los accionantes no presentaron un argumento claro sobre los derechos supuestamente violado...



## Querys

In [7]:
pregunta = df.iloc[20]['query']
print(pregunta)

¿Cuál fue la razón principal por la que la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N. 2511-18-EP presentada por Juan Ricardo Cedillo Rizo?


## Embeddings

In [ ]:
#esto no iría en el notebook, solo para montar drive en el colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ruta_embeddings = '/content/drive/MyDrive/Colab Notebooks/embeddings_semanticos.npy' #modificar ruta para local

if os.path.exists(ruta_embeddings):
    doc_embs = np.load(ruta_embeddings)
    print("embeddings cargados")
else:
    print(f"generando embeddings")
    doc_embs = modelo.encode(corpus_original)

    np.save(ruta_embeddings, doc_embs)
    print(f"embeddings guardados")

print("Dimensiones finales de los Embeddings:", doc_embs.shape)

generando embeddings
embeddings guardados
Dimensiones finales de los Embeddings: (22458, 384)


In [17]:
query_emb = modelo.encode(pregunta)

## Similitud coseno

In [18]:
coseno_scores = util.cos_sim(query_emb, doc_embs)[0]
resultados = sorted(zip(corpus_original, coseno_scores), key=lambda x: x[1], reverse=True)[:20]
for doc, score in resultados:
  print(f"{score:.4f} -> {doc}")

0.8553 -> La Corte Constitucional inadmitió la acción extraordinaria de protección No. 0007-11-EP porque los accionantes, Carlos Alberto Rodríguez García y Merci Marlene Chapa Veletanga, confundieron el objeto de esta acción al pretender que la Corte reevaluara el fondo de un asunto ya resuelto por la justicia ordinaria, el cual les fue desfavorable. Adicionalmente, no justificaron el cumplimiento de los requisitos de admisibilidad estipulados en el artículo 62, numerales 2 y 3, de la Ley Orgánica de Garantías Jurisdiccionales y Control Constitucional. La decisión se fundamenta en el artículo 437 de la Constitución, que exige que la acción se presente contra resoluciones firmes y que se demuestre la violación del debido proceso u otros derechos constitucionales durante el juzgamiento. La Sala de Admisión, en aplicación del artículo 62 de la Ley Orgánica mencionada y los artículos 12 y 35 del Reglamento de Sustanciación de Procesos, determinó la inadmisión del recurso.
0.8553 -> La Cort

## Búsqueda

In [22]:
def busqueda_transformers(pregunta_usuario, top_n=5):
  print(f"<--- Buscando: '{pregunta_usuario}' --->")

  pregunta_vector = modelo.encode([pregunta_usuario])
  similitudes = cosine_similarity(pregunta_vector, doc_embs).flatten()
  indices_top = similitudes.argsort()[-top_n:][::-1]

  print("Resultados encontrados:\n")
  resultados = []
  for i in indices_top:
    print(f"  Puntaje de Similitud: {similitudes[i]:.4f}")
    print(f"  Texto: {corpus_original[i][:250]}...\n")
    resultados.append(i)
  return resultados, similitudes

In [23]:
pregunta_de_prueba = df.iloc[20]['query']
_ = busqueda_transformers(pregunta_de_prueba)

pregunta_personalizada = "¿Qué pasa con las obligaciones del vendedor?"
_ = busqueda_transformers(pregunta_personalizada)

<--- Buscando: '¿Cuál fue la razón principal por la que la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N. 2511-18-EP presentada por Juan Ricardo Cedillo Rizo?' --->
Resultados encontrados:

  Puntaje de Similitud: 0.8553
  Texto: La Corte Constitucional inadmitió la acción extraordinaria de protección No. 0007-11-EP porque los accionantes, Carlos Alberto Rodríguez García y Merci Marlene Chapa Veletanga, confundieron el objeto de esta acción al pretender que la Corte reevaluar...

  Puntaje de Similitud: 0.8553
  Texto: La Corte Constitucional inadmitió la acción extraordinaria de protección Nro. 1795-10-EP debido a que el accionante, Ing. Moisés Fernando Tacle Galárraga, no cumplió con los requisitos de admisibilidad establecidos en el artículo 62 de la Ley Orgánic...

  Puntaje de Similitud: 0.8547
  Texto: La Corte Constitucional inadmitió la acción extraordinaria de protección No. 0344-11-EP debido a que el accionante, señor César Augusto Guerrero